In [1]:
from function import *
from data_preparation import *
from evaluation import *

from openpyxl import Workbook
import pandas as pd
import numpy as np
import operator

from rdkit import Chem
from rdkit.Chem import AllChem

# Pre-prepare Data (Data Transformation)

In [2]:
transform_data(output_file_name='../dataset/keck_complete.csv')

# Analysis

In [3]:
complete_df = pd.read_csv('../dataset/keck_complete.csv')

print complete_df.dtypes

Molecule                   object
SMILES                     object
Fingerprints               object
Keck_Pria_AS_Retest         int64
Keck_Pria_FP_data           int64
Keck_Pria_Continuous      float64
Keck_RMI_cdd              float64
FP counts % inhibition    float64
dtype: object


In [4]:
cnt_pria_retest = 0
cnt_pria_fp = 0
cnt_rmi_cdd = 0

cnt_dict = {}
for ix, row in complete_df.iterrows():
    cnt = 0
    if row['Keck_Pria_AS_Retest'] == 1:
        cnt_pria_retest += 1
        cnt += 1
    if row['Keck_Pria_FP_data'] == 1:
        cnt_pria_fp += 1
        cnt += 1
    if row['Keck_RMI_cdd'] == 1:
        cnt_rmi_cdd += 1
        cnt += 1
    if cnt not in cnt_dict.keys():
        cnt_dict[cnt] = 0
    cnt_dict[cnt] += 1

print 'pria retest active: {}\tpria fp active: {}\trmi cdd active: {}'.format(cnt_pria_retest, cnt_pria_fp, cnt_rmi_cdd)
print cnt_dict


print
analysis(complete_df)

pria retest active: 79	pria fp active: 24	rmi cdd active: 230
{0: 72094, 1: 325, 2: 4}

retest: 0, fp: 0, rmi: 0 	--- 49489
retest: 0, fp: 0, rmi: nan 	--- 22605
retest: 0, fp: 1, rmi: 0 	--- 19
retest: 0, fp: 1, rmi: 1 	--- 3
retest: 1, fp: 0, rmi: 0 	--- 58
retest: 1, fp: 0, rmi: nan 	--- 20
retest: 0, fp: 1, rmi: nan 	--- 1
retest: 0, fp: 0, rmi: 1 	--- 227
retest: 1, fp: 1, rmi: nan 	--- 1


# Split data into k-fold

In [5]:
k = 3
directory = '../dataset/fixed_dataset/fold_{}/'.format(k)
file_list = []
for i in range(k):
    file_list.append('file_{}.csv'.format(i))
greedy_multi_splitting(complete_df, k, directory=directory, file_list=file_list)

9
(24144, 8)
(24140, 8)
(24139, 8)


In [6]:
k = 4
directory = '../dataset/fixed_dataset/fold_{}/'.format(k)
file_list = []
for i in range(k):
    file_list.append('file_{}.csv'.format(i))
greedy_multi_splitting(complete_df, k, directory=directory, file_list=file_list)

9
(18105, 8)
(18106, 8)
(18106, 8)
(18106, 8)


# Merge data from splitting folds to form training and testing

In [7]:
dtype_list = {'Molecule': np.str,
              'SMILES':np.str,
              'Fingerprints': np.str,
              'Keck_Pria_AS_Retest': np.int64,
              'Keck_Pria_FP_data': np.int64,
              'Keck_Pria_Continuous': np.float64,
              'Keck_RMI_cdd': np.float64}
output_file_list = [directory + f_ for f_ in file_list]
print output_file_list[:3]
train_pd = read_merged_data(output_file_list[:3])
print output_file_list[3]
test_pd = read_merged_data([output_file_list[3]])

['../dataset/fixed_dataset/fold_4/file_0.csv', '../dataset/fixed_dataset/fold_4/file_1.csv', '../dataset/fixed_dataset/fold_4/file_2.csv']
../dataset/fixed_dataset/fold_4/file_3.csv


In [ ]:
print 'This is training set'
analysis(train_pd)
print
print 'This is test set'
analysis(test_pd)

This is training set
retest: 0, fp: 0, rmi: 0 	--- 37117
retest: 0, fp: 0, rmi: nan 	--- 16954
retest: 1, fp: 0, rmi: 0 	--- 43
retest: 0, fp: 1, rmi: 1 	--- 2
retest: 0, fp: 0, rmi: 1 	--- 170
retest: 0, fp: 1, rmi: 0 	--- 14
retest: 0, fp: 1, rmi: nan 	--- 1
retest: 1, fp: 0, rmi: nan 	--- 15
retest: 1, fp: 1, rmi: nan 	--- 1

This is test set
retest: 0, fp: 0, rmi: 0 	--- 12372
retest: 0, fp: 0, rmi: nan 	--- 5651
retest: 1, fp: 0, rmi: nan 	--- 5
retest: 0, fp: 1, rmi: 1 	--- 1
retest: 1, fp: 0, rmi: 0 	--- 15
retest: 0, fp: 0, rmi: 1 	--- 57
retest: 0, fp: 1, rmi: 0 	--- 5


# Test feature- and label- extraction

In [ ]:
print train_pd.dtypes

X_train, y_train = extract_feature_and_label(train_pd,
                                             feature_name='Fingerprints',
                                             label_name_list=['Keck_Pria_AS_Retest', 'Keck_Pria_FP_data'])
X_test, y_test = extract_feature_and_label(test_pd,
                                           feature_name='Fingerprints',
                                           label_name_list=['Keck_Pria_AS_Retest', 'Keck_Pria_FP_data'])

Molecule                   object
SMILES                     object
Fingerprints               object
Keck_Pria_AS_Retest         int64
Keck_Pria_FP_data           int64
Keck_Pria_Continuous      float64
Keck_RMI_cdd              float64
FP counts % inhibition    float64
dtype: object


# Test evaluation metrics

In [ ]:
y_actual = np.eye(4)
y_pred = np.eye(4)
y_pred[2] = np.array([0, 0, 1, 1])
y_pred[3] = np.array([0, 0, 1, 0])

for i in range(4):
    print roc_auc_multi(y_actual, y_pred, [i], np.mean)
    print roc_auc_multi(y_actual, y_pred, [i], np.median)
    print enrichment_factor_multi(y_actual, y_pred, 0.5)
    print